<a href="https://colab.research.google.com/github/Shameerisb/Cuda_Convolution/blob/main/Git_Mnist_in_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%capture
#main Repo
!git clone https://github.com/Shameerisb/Cuda_Convolution.git

# Clone the cnpy repository
!git clone https://github.com/rogersce/cnpy.git

# Build the cnpy library
!cd cnpy && mkdir build && cd build && cmake .. && make

fatal: destination path 'Cuda_Convolution' already exists and is not an empty directory.
fatal: destination path 'cnpy' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists


### Download Mnist and save verification files.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.datasets import mnist
import os


input_index = int(input("Please enter the input index: "))  # Convert the input to an integer


(X_train,Y_train),(X_test,Y_test) = mnist.load_data()
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')

## data normalization
X_train=X_train/255
X_test=X_test/255

np.save(f'/content/Cuda_Convolution/Data/Input_Data', X_train)
print("Mnist  Data downloded and saved")




################# Weights_Biases ###################

# Load the pre-trained model
model = load_model('/content/Cuda_Convolution/Data/Model/mnist_model_Basic.keras')

def remove_none_shape(shape):
    """Remove None dimensions from the shape tuple."""
    return [dim for dim in shape if dim is not None]

# Directory to save .npy files
save_dir = '/content/Cuda_Convolution/Data/Mnist_images/Weights_and_Biases'

# Iterate over each layer in the model
for i, layer in enumerate(model.layers):
    i = i + 1

    layer_name = layer.name
    layer_weights = layer.get_weights()

    # Get the input and output shape of the layer
    input_shape = remove_none_shape(layer.input.shape) if layer.input is not None else []
    output_shape = remove_none_shape(layer.output.shape) if layer.output is not None else []

    if input_shape:
        # Convert shapes to numpy arrays
        input_shape_np = np.array(input_shape, dtype=np.int64)
        # Save input shape
        np.save(os.path.join(save_dir, f'layer_{i}_input_shape.npy'), input_shape_np)
    else:
        # Handle case where input_shape is empty
        print(f"Layer {i} input_shape is empty and was not saved.")

    if output_shape:
        # Convert shapes to numpy arrays
        output_shape_np = np.array(output_shape, dtype=np.int64)
        # Save output shape
        np.save(os.path.join(save_dir, f'layer_{i}_output_shape.npy'), output_shape_np)
    else:
        # Handle case where output_shape is empty
        print(f"Layer {i} output_shape is empty and was not saved.")

    if layer_weights:  # Check if the layer has weights
        weights, biases = layer_weights

        if i < 4:
            weights = np.transpose(weights, (3, 0, 1, 2))

        # print(weights.shape)
        # Save weights and biases
        np.save(os.path.join(save_dir, f'layer_{i}_weights.npy'), weights)
        np.save(os.path.join(save_dir, f'layer_{i}_biases.npy'), biases)

print("Weights, biases, and shapes for all layers saved to individual .npy files.")


################# Activation_Maps ###################

# Create a new input layer
input_layer = Input(shape=(28, 28, 1))

# Create a new model that outputs the activations of each layer
x = input_layer
layer_outputs = []
for layer in model.layers:
    x = layer(x)
    layer_outputs.append(x)

activation_model = Model(inputs=input_layer, outputs=layer_outputs)
sample_input = X_train[input_index].reshape(1, 28, 28, 1)

# Get the activations
activations = activation_model.predict(sample_input)

# Define the base path for saving the activation maps
base_path = '/content/Cuda_Convolution/Data/Mnist_images/Verification_data/'

# Plot the activation maps and save them
for layer_index, activation in enumerate(activations):
    # Preprocess the activation maps
    activation_maps = np.squeeze(activation)  # Remove single-dimensional entries

    # Save the activation maps
    np.save(f"{base_path}Input_{input_index}_L_{layer_index + 1}.npy", activation_maps)

    # print(f"Layer {layer_index + 1} activation shape: {activation_maps.shape}")

print("Activation maps saved successfully.")


Please enter the input index: 1
Mnist  Data downloded and saved
Weights, biases, and shapes for all layers saved to individual .npy files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
Activation maps saved successfully.


### Convolution_main


In [ ]:
# Compile the Files

!nvcc -o Mnist_1st_layer  \
'/content/cnpy/cnpy.cpp'\
'/content/Cuda_Convolution/Source_Code/Mnist_main.cu' \
'/content/Cuda_Convolution/Source_Code/utils.cpp' \
'/content/Cuda_Convolution/Source_Code/load_npy.cpp' \
'/content/Cuda_Convolution/Source_Code/Kernels.cu' \
-I/usr/include \
-I/usr/local/cuda/include \
-L/usr/lib/x86_64-linux-gnu \
-lcudnn -lcuda -Icnpy -lz


profiling = input("Do you want to perform profiling? (y/n): ")

if profiling == 'y':
  !nvprof --print-gpu-trace ./Mnist_1st_layer
else:
  !./Mnist_1st_layer
